In [3]:
import requests
import json
import urllib.parse

In [35]:
import requests

# Define the API endpoint
api_url = 'http://mygene.info/v3/query'

# Define the search term
search_term = 'cardiovascular disease'

# Define the API parameters
params = {
    'q': search_term,
    'fields': 'symbol,name,summary',
    'species': 'human'
}

# Send the GET request to the API and retrieve the response
response = requests.get(api_url, params=params)

# Check if the request was successful
if response.ok:
    # Retrieve the JSON data from the response
    data = response.json()
    
    genes = []
    # Extract gene information from the response
    for result in data['hits']:
        gene_symbol = result['symbol']
        genes.append(gene_symbol)

else:
    # Print an error message if the request was unsuccessful
    print('Error:', response.status_code, response.text)

print(genes)

['CDKN2B-AS1', 'MGP', 'TNNI3', 'CYSLTR2', 'ACE2', 'HEY2', 'NPPB', 'KLF5', 'HEY1', 'TBXAS1']


In [36]:
# Define the API endpoint
api_url = 'https://rest.ensembl.org/xrefs/symbol/homo_sapiens/'

# Define the API parameters
params = {
    'content-type': 'application/json',
    'object_type': 'gene',
    'ensembl_release': 'GRCh38'
}

ensembl_ids = []
for gene in genes:
    # Send the GET request to the API and retrieve the response
    response = requests.get(api_url + gene, params=params)

    # Check if the request was successful
    if response.ok:
        # Retrieve the JSON data from the response
        data = response.json()
        
        # Extract the Ensembl gene ID from the response
        for result in data:
            ensembl_ids.append(result['id'])
    else:
        # Print an error message if the request was unsuccessful
        print('Error:', response.status_code, response.text)

print(ensembl_ids)

['ENSG00000240498', 'ENSG00000111341', 'ENSG00000129991', 'LRG_432', 'ENSG00000152207', 'ENSG00000130234', 'ENSG00000135547', 'ENSG00000120937', 'ENSG00000102554', 'ENSG00000164683', 'ENSG00000059377', 'LRG_579']


In [37]:
# Use Ensembl to find orthologues

orthologues = {}

for ensembl in ensembl_ids:

# TODO: add ability to query multiple genes, or a list of genes related to some disease
# OR all genes in a subset of species
    # Define the API endpoint
    api_url = 'https://rest.ensembl.org/homology/id/'

    # Define the Ensembl gene ID
    ensembl_id = ensembl

    # Define the API parameters
    params = {
        'content-type': 'application/json',
        'type': 'orthologues',
        'target_taxon': '10090' # Mouse
    }

    # Send the GET request to the API and retrieve the response
    response = requests.get(api_url + ensembl_id, params=params)

    #print(data["data"][0]["homologies"])

    # Check if the request was successful
    if response.ok:
        # Retrieve the JSON data from the response
        data = response.json()
        
        # Extract the orthologous gene IDs from the response
        
        for homology in data["data"][0]["homologies"]:
            #if homology["target"]["species"] in species_of_interest:  # orthologues within the species of interest
            if(homology['type'] == 'ortholog_one2one'):
                target_gene = homology["target"]["id"]
                target_species = homology["target"]["species"]
                target_orthology = homology["type"]
                print(target_gene)
                print(target_species)
                print(target_orthology)
                print()
                ortholog = [target_gene, target_species]
                orthologues[ensembl] = ortholog

print(orthologues)
print(len(orthologues))

        # data is a dictionary with only one key, which is "data"
        # data["data"] is list with only one element
        # data["data"][0] is a dictionary with the following keys:
        #  "id", and the value is just a string with the gene ID (e.g. "ENSG00000157764")
        # "homologies", and the value is a list with 227 elements, iterating over the homologues (orthologues, paralogues)

        #print(len(data["data"][0]["homologies"]))
        #print(data["data"][0]["homologies"][0]['source'])

ENSMUSG00000030218
mus_musculus
ortholog_one2one

ENSMUSG00000035458
mus_musculus
ortholog_one2one

ENSMUSG00000015405
mus_musculus
ortholog_one2one

ENSMUSG00000019789
mus_musculus
ortholog_one2one

ENSMUSG00000029019
mus_musculus
ortholog_one2one

ENSMUSG00000005148
mus_musculus
ortholog_one2one

ENSMUSG00000040289
mus_musculus
ortholog_one2one

ENSMUSG00000029925
mus_musculus
ortholog_one2one

{'ENSG00000111341': ['ENSMUSG00000030218', 'mus_musculus'], 'ENSG00000129991': ['ENSMUSG00000035458', 'mus_musculus'], 'ENSG00000130234': ['ENSMUSG00000015405', 'mus_musculus'], 'ENSG00000135547': ['ENSMUSG00000019789', 'mus_musculus'], 'ENSG00000120937': ['ENSMUSG00000029019', 'mus_musculus'], 'ENSG00000102554': ['ENSMUSG00000005148', 'mus_musculus'], 'ENSG00000164683': ['ENSMUSG00000040289', 'mus_musculus'], 'ENSG00000059377': ['ENSMUSG00000029925', 'mus_musculus']}
8


In [38]:
# Use UniProt to find the proteins produced by the orthologues
proteins = []
for orthologue in orthologues:
    print(orthologue)
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=json&query=%28{orthologue[1]}%20{orthologue[0]}%29"
    response = requests.get(uniprot_url)
    # convert response.content from bytes to dictionary
    data = json.loads(response.content)
    # If the data is empty, skip it
    # data is a dictionary with only one key, which is "results"
    # data["results"] is a list with 3 elements, one for each protein
    # data["results"][0] is a dictionary with these entries: 'entryType', 'primaryAccession', 'uniProtkbId', 'entryAudit', 'annotationScore', 'organism', 'proteinExistence', 'proteinDescription', 'genes', 'comments', 'features', 'keywords', 'references', 'uniProtKBCrossReferences', 'sequence', 'extraAttributes'
    if(len(data["results"]) == 0):
        continue
    elif(orthologue[1] in species_of_interest):
    # check if organism match with the orthologue input, print both
    # print(data["results"][0]["organism"]["scientificName"], orthologue[1])
    # print(data["results"][0]["primaryAccession"], data["results"][0]["uniProtkbId"])
        #print(orthologue[0], orthologue[1])
        print(data['results'][0]['comments'][5])
        print('============================')
        protein_species = (data["results"][0]["primaryAccession"], data["results"][0]["organism"]["taxonId"], data['results'][0]['genes'][0]['geneName']['value'])
        proteins.append(protein_species)

print(proteins)

ENSG00000111341


KeyError: 'results'